In [ ]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import wandb
import numpy as np
import os
import pandas as pd
import zipfile
import random
from tqdm import tqdm
from google.colab import files
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# # Define the model architecture
# class Encoder(nn.Module):
#     def __init__(self, input_size,embedding_size, hidden_size,batch_size,encoder_num_layers, cell_type, bidirectional, dropout):
#         super(Encoder, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(input_size, embedding_size)
#         self.dropout = nn.Dropout(dropout)
#         self.batch_size=batch_size
#         self.embedding_size=embedding_size
#         self.encoder_num_layers=encoder_num_layers
#         self.cell_type = cell_type
#         self.bidirectional=bidirectional



#         if cell_type == "RNN":
#             self.rnn = nn.RNN(self.embedding_size, self.hidden_size,self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
#         elif cell_type == "LSTM":
#             self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
#         elif cell_type == "GRU":
#             self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)




#     def forward(self, input_seq, hidden):
#         embedded = self.dropout((self.embedding(input_seq.long())).view(-1,self.batch_size, self.embedding_size))
#         outputs, hidden = self.rnn(embedded,hidden)


#         if self.bidirectional:
#             if self.cell_type == "LSTM":
#                 # Dividing the hidden state into parts for each direction
#                 hidden_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)
#                 cell_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

#                 # Combining the hidden and cell states by taking their average
#                 hidden = (torch.add(hidden_state[0], hidden_state[1]) / 2, torch.add(cell_state[0], cell_state[1]) / 2)
#             else:
#                 # Dividing the hidden state into parts for each direction
#                 hidden = hidden.view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

#                 # Combining the hidden states by taking their average
#                 hidden = torch.add(hidden[0], hidden[1]) / 2

#             # Splitting the output tensor into parts for each direction
#             split_tensor = torch.split(outputs, self.hidden_size, dim=-1)

#             # Combining the outputs by taking their average
#             outputs = torch.add(split_tensor[0], split_tensor[1]) / 2


#         return outputs, hidden



#     def initHidden(self):
#             num_directions = 2 if self.bidirectional else 1  # For bidirectional, set to 2, otherwise 1
#             if self.cell_type == "LSTM":
#                 return (torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device),
#                         torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device))
#             else:
#                 return torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device)

# class Decoder(nn.Module):
#     def __init__(self, output_size, embedding_size, hidden_size, batch_size, decoder_num_layers, cell_type, dropout, MAX_LENGTH):
#         super(Decoder, self).__init__()
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.batch_size = batch_size
#         self.embedding_size = embedding_size
#         self.decoder_num_layers = decoder_num_layers
#         self.MAX_LENGTH = MAX_LENGTH
#         self.cell_type = cell_type
#         self.dropout=dropout

#         self.embedding = nn.Embedding(output_size, embedding_size)
#         self.dropout = nn.Dropout(dropout)
#         if cell_type == "RNN":
#             self.rnn = nn.RNN(embedding_size, hidden_size, self.decoder_num_layers,dropout=dropout)
#         elif cell_type == "LSTM":
#             self.rnn = nn.LSTM(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)
#         elif cell_type == "GRU":
#             self.rnn = nn.GRU(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)


#         self.out = nn.Linear(hidden_size, output_size)

#         self.softmax = nn.LogSoftmax(dim=2)

#     def forward(self, input, hidden):
#         output = self.embedding(input.long()).view(-1,self.batch_size, self.embedding_size)

#         output = F.relu(output)

#         output, hidden = self.rnn(output, hidden)

#         output = self.softmax(self.out(output))
#         return output, hidden




In [ ]:
# Define the model architecture
class Encoder(nn.Module):
    """
    Encoder module of the sequence-to-sequence model.

    Parameters:
        input_size (int): The size of the input vocabulary.
        embedding_size (int): The dimension of the embedding layer.
        hidden_size (int): The size of the hidden state of the RNN.
        batch_size (int): The size of the batches.
        encoder_num_layers (int): The number of layers in the encoder.
        cell_type (str): The type of RNN cell: "RNN", "LSTM", or "GRU".
        bidirectional (bool): Whether to use bidirectional RNN.
        dropout (float): Dropout rate.

    Inputs:
        input_seq (tensor): Input sequence tensor.
        hidden (tensor): Initial hidden state tensor.

    Outputs:
        outputs (tensor): Encoder outputs tensor.
        hidden (tensor): Final hidden state tensor.
    """

    def __init__(self, input_size, embedding_size, hidden_size, batch_size, encoder_num_layers, cell_type, bidirectional, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.encoder_num_layers = encoder_num_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional

        if cell_type == "RNN":
            self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)

    def forward(self, input_seq, hidden):
        embedded = self.dropout((self.embedding(input_seq.long())).view(-1, self.batch_size, self.embedding_size))
        outputs, hidden = self.rnn(embedded, hidden)

        if self.bidirectional:
            if self.cell_type == "LSTM":
                # Dividing the hidden state into parts for each direction
                hidden_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)
                cell_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

                # Combining the hidden and cell states by taking their average
                hidden = (torch.add(hidden_state[0], hidden_state[1]) / 2, torch.add(cell_state[0], cell_state[1]) / 2)
            else:
                # Dividing the hidden state into parts for each direction
                hidden = hidden.view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

                # Combining the hidden states by taking their average
                hidden = torch.add(hidden[0], hidden[1]) / 2

            # Splitting the output tensor into parts for each direction
            split_tensor = torch.split(outputs, self.hidden_size, dim=-1)

            # Combining the outputs by taking their average
            outputs = torch.add(split_tensor[0], split_tensor[1]) / 2

        return outputs, hidden

    def initHidden(self):
        num_directions = 2 if self.bidirectional else 1  # For bidirectional, set to 2, otherwise 1
        if self.cell_type == "LSTM":
            return (torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device),
                    torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device))
        else:
            return torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device)


class Decoder(nn.Module):
    """
    Decoder module of the sequence-to-sequence model.

    Parameters:
        output_size (int): The size of the output vocabulary.
        embedding_size (int): The dimension of the embedding layer.
        hidden_size (int): The size of the hidden state of the RNN.
        batch_size (int): The size of the batches.
        decoder_num_layers (int): The number of layers in the decoder.
        cell_type (str): The type of RNN cell: "RNN", "LSTM", or "GRU".
        dropout (float): Dropout rate.
        MAX_LENGTH (int): Maximum length of the input sequence.

    Inputs:
        input (tensor): Input tensor.
        hidden (tensor): Hidden state tensor.

    Outputs:
        output (tensor): Output tensor.
        hidden (tensor): Hidden state tensor.
    """

    def __init__(self, output_size, embedding_size, hidden_size, batch_size, decoder_num_layers, cell_type, dropout, MAX_LENGTH):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.decoder_num_layers = decoder_num_layers
        self.MAX_LENGTH = MAX_LENGTH
        self.cell_type = cell_type
        self.dropout = dropout

        self.embedding = nn.Embedding(output_size, embedding_size)
        self.dropout = nn.Dropout(dropout)
        if cell_type == "RNN":
            self.rnn = nn.RNN(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output = self.embedding(input.long()).view(-1, self.batch_size, self.embedding_size)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output))
        return output, hidden


In [ ]:
# class AttentionDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, embedding_size, num_layers,
#                  cell_type, dropout, batch_size, max_length):
#         super(AttentionDecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.cell_type = cell_type
#         self.batch_size = batch_size
#         self.embedding_size = embedding_size
#         self.max_length = max_length
#         self.dropout = dropout

#         self.embedding = nn.Embedding(output_size, embedding_size)
#         self.dropout = nn.Dropout(self.dropout)
#         self.attention = nn.Linear(hidden_size + embedding_size, self.max_length)
#         self.attention_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

#         if self.cell_type == "GRU":
#             self.rnn = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)
#         elif self.cell_type == "LSTM":
#             self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers)
#         else:
#             self.rnn = nn.RNN(hidden_size, hidden_size, num_layers=num_layers)

#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=2)

#     def forward(self, input, hidden, encoder_outputs): #input shape (1, batch_size)
#         embedded = self.embedding(input.long()).view(-1, self.batch_size, self.embedding_size)
#         # embedded shape (1, batch_size, embedding_size)
#         embedded = F.relu(embedded)

#         # Compute attention scores
#         if self.cell_type == "LSTM":
#             attn_hidden = torch.mean(hidden[0], dim=0)
#         else:
#             attn_hidden = torch.mean(hidden, dim = 0)
#         attn_scores = self.attention(torch.cat((embedded, attn_hidden.unsqueeze(0)), dim=2)) # attn_scores shape (1, batch_size, max_length)

#         attn_weights = F.softmax(attn_scores, dim=-1)  # attn_scores shape (1, 16, 25)


#         # Apply attention weights to encoder outputs
#         # print(encoder_outputs.transpose(0, 1).shape)
#         attn_applied = torch.bmm(attn_weights.transpose(0, 1), encoder_outputs.transpose(0, 1))

#         # Combine attention output and embedded input
#         combined = torch.cat((embedded, attn_applied.transpose(0, 1)), dim=2)
#         combined = self.attention_combine(combined)
#         combined = F.relu(combined) # shape (1, batch_size, hidden_size)

#         # Run through the RNN
#         output, hidden = self.rnn(combined, hidden)
#         # output shape: (1, batch_size, hidden_size)

#         # Pass through linear layer and softmax activation
#         output = self.out(output)  # shape: (1, batch_size, output_size)
#         output = self.softmax(output)
#         return output, hidden, attn_weights.transpose(0, 1)

In [ ]:
class AttentionDecoderRNN(nn.Module):
    """
    Attention-based Decoder module of the sequence-to-sequence model.

    Parameters:
        hidden_size (int): The size of the hidden state of the RNN.
        output_size (int): The size of the output vocabulary.
        embedding_size (int): The dimension of the embedding layer.
        num_layers (int): The number of layers in the RNN.
        cell_type (str): The type of RNN cell: "RNN", "LSTM", or "GRU".
        dropout (float): Dropout rate.
        batch_size (int): The size of the batches.
        max_length (int): Maximum length of the input sequence.

    Inputs:
        input (tensor): Input tensor.
        hidden (tensor): Hidden state tensor.
        encoder_outputs (tensor): Encoder outputs tensor.

    Outputs:
        output (tensor): Output tensor.
        hidden (tensor): Hidden state tensor.
        attn_weights (tensor): Attention weights tensor.
    """

    def __init__(self, hidden_size, output_size, embedding_size, num_layers,
                 cell_type, dropout, batch_size, max_length):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.max_length = max_length
        self.dropout = dropout

        self.embedding = nn.Embedding(output_size, embedding_size)
        self.dropout = nn.Dropout(self.dropout)
        self.attention = nn.Linear(hidden_size + embedding_size, self.max_length)
        self.attention_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

        if self.cell_type == "GRU":
            self.rnn = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)
        elif self.cell_type == "LSTM":
            self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers)
        else:
            self.rnn = nn.RNN(hidden_size, hidden_size, num_layers=num_layers)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input.long()).view(-1, self.batch_size, self.embedding_size)
        embedded = F.relu(embedded)

        # Compute attention scores
        if self.cell_type == "LSTM":
            attn_hidden = torch.mean(hidden[0], dim=0)
        else:
            attn_hidden = torch.mean(hidden, dim=0)
        attn_scores = self.attention(torch.cat((embedded, attn_hidden.unsqueeze(0)), dim=2))

        attn_weights = F.softmax(attn_scores, dim=-1)

        # Apply attention weights to encoder outputs
        attn_applied = torch.bmm(attn_weights.transpose(0, 1), encoder_outputs.transpose(0, 1))

        # Combine attention output and embedded input
        combined = torch.cat((embedded, attn_applied.transpose(0, 1)), dim=2)
        combined = self.attention_combine(combined)
        combined = F.relu(combined)

        # Run through the RNN
        output, hidden = self.rnn(combined, hidden)

        # Pass through linear layer and softmax activation
        output = self.out(output)
        output = self.softmax(output)
        return output, hidden, attn_weights.transpose(0, 1)


In [ ]:
# def Datasetretrival(pathofzip, folder_name):
#     # Define the path to your zip file
#     zip_file_path = pathofzip

#     # Extract the contents of the zip file
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall('extracted_data')

#     # Define the path to the extracted data directory
#     extracted_data_dir = 'extracted_data/aksharantar_sampled'
#     contents = os.listdir(extracted_data_dir)

#     # Initialize empty lists for train, test, and validation datasets
#     train_datasets = []
#     test_datasets = []
#     val_datasets = []

#  # Load train, test, and validation CSV files from the specified folder
#     folder_path = os.path.join(extracted_data_dir, folder_name)

#     # List all files in the folder
#     folder_files = os.listdir(folder_path)


#     # Filter files with the specified folder name as prefix
#     foldername_prefix = folder_name + "_"
#     folder_files_with_prefix = [file for file in folder_files if file.startswith(foldername_prefix)]

#     for file in folder_files_with_prefix:
#         file_path = os.path.join(folder_path, file)
#         if 'train' in file:
#             train_datasets.append(pd.read_csv(file_path,header=None))
#         elif 'test' in file:
#             test_datasets.append(pd.read_csv(file_path,header=None))
#         elif 'val' in file:
#             val_datasets.append(pd.read_csv(file_path,header=None))

#     # Concatenate the loaded dataframes to create single train, test, and validation datasets
#     train_dataset = pd.concat(train_datasets, ignore_index=True)
#     test_dataset = pd.concat(test_datasets, ignore_index=True)
#     val_dataset = pd.concat(val_datasets, ignore_index=True)


#     return train_dataset, val_dataset, test_dataset

In [ ]:
def Datasetretrival(pathofzip, folder_name):
    """
    Retrieve datasets from a zip file.

    Parameters:
        pathofzip (str): Path to the zip file containing the datasets.
        folder_name (str): Name of the folder containing the datasets.

    Returns:
        train_dataset (DataFrame): DataFrame containing the training dataset.
        val_dataset (DataFrame): DataFrame containing the validation dataset.
        test_dataset (DataFrame): DataFrame containing the test dataset.
    """
    # Extract the contents of the zip file
    with zipfile.ZipFile(pathofzip, 'r') as zip_ref:
        zip_ref.extractall('extracted_data')

    # Define the path to the extracted data directory
    extracted_data_dir = 'extracted_data/aksharantar_sampled'
    folder_path = os.path.join(extracted_data_dir, folder_name)

    # Initialize empty lists for train, test, and validation datasets
    train_datasets = []
    test_datasets = []
    val_datasets = []

    # List all files in the folder
    folder_files = os.listdir(folder_path)

    # Filter files with the specified folder name as prefix
    foldername_prefix = folder_name + "_"
    folder_files_with_prefix = [file for file in folder_files if file.startswith(foldername_prefix)]

    # Load train, test, and validation CSV files from the specified folder
    for file in folder_files_with_prefix:
        file_path = os.path.join(folder_path, file)
        if 'train' in file:
            train_datasets.append(pd.read_csv(file_path, header=None))
        elif 'test' in file:
            test_datasets.append(pd.read_csv(file_path, header=None))
        elif 'val' in file:
            val_datasets.append(pd.read_csv(file_path, header=None))

    # Concatenate the loaded dataframes to create single train, test, and validation datasets
    train_dataset = pd.concat(train_datasets, ignore_index=True)
    test_dataset = pd.concat(test_datasets, ignore_index=True)
    val_dataset = pd.concat(val_datasets, ignore_index=True)

    return train_dataset, val_dataset, test_dataset


In [ ]:
# def vecorizeddata(data_pairs,index2char, char2index,MAX_LENGTH):




#     # Adding in the main index2char and char2index dictionary
#     for word_pair in data_pairs:

#         for char in word_pair[0]:
#             if char not in  char2index:
#                 char2index[char] = len(char2index)
#                 index2char[len(index2char)] = char




#         for char in word_pair[1]:
#             if char not in  char2index:
#                 char2index[char] = len(char2index)
#                 index2char[len(index2char)] = char




#     SOS_token = 0
#     EOS_token = 1
#     PAD_token = 2

#     vec_pair_list = []
#     for word_pair in data_pairs:


#         vec_1 = []
#         for char in word_pair[0]:
#             vec_1.append(char2index[char])

#         wordvec_1 = vec_1
#         wordvec_1.append(EOS_token)

#         for i in range(MAX_LENGTH  - len(word_pair[0])):
#             wordvec_1.append(PAD_token)
#         wordvec_1 = torch.LongTensor(wordvec_1)
#         eng_vec = wordvec_1


#         vec_2 = []
#         for char in word_pair[1]:
#             vec_2.append(char2index[char])

#         wordvec_2 = vec_2
#         wordvec_2.append(EOS_token)

#         for i in range(MAX_LENGTH  - len(word_pair[1])):
#             wordvec_2.append(PAD_token)
#         wordvec_2 = torch.LongTensor(wordvec_2)
#         guj_vec = wordvec_2


#         vec_pair = (eng_vec, guj_vec)
#         vec_pair_list.append(vec_pair)

#     return vec_pair_list,char2index, index2char


In [ ]:
import torch

def vecorizeddata(data_pairs, index2char, char2index, MAX_LENGTH):
    """
    Vectorize the input data pairs for training.

    Parameters:
        data_pairs (list): List of tuples containing pairs of input-output data.
        index2char (dict): Dictionary mapping indices to characters.
        char2index (dict): Dictionary mapping characters to indices.
        MAX_LENGTH (int): Maximum length of input sequences.

    Returns:
        vec_pair_list (list): List of tuples containing vectorized input-output pairs.
        char2index (dict): Updated character to index dictionary.
        index2char (dict): Updated index to character dictionary.
    """
    # Adding characters to the main index2char and char2index dictionary
    for word_pair in data_pairs:
        for char in word_pair[0]:
            if char not in char2index:
                char2index[char] = len(char2index)
                index2char[len(index2char)] = char

        for char in word_pair[1]:
            if char not in char2index:
                char2index[char] = len(char2index)
                index2char[len(index2char)] = char

    SOS_token = 0
    EOS_token = 1
    PAD_token = 2

    vec_pair_list = []
    for word_pair in data_pairs:
        # Vectorize English word
        eng_vec = [char2index[char] for char in word_pair[0]]
        eng_vec.append(EOS_token)
        eng_vec += [PAD_token] * (MAX_LENGTH - len(word_pair[0]))
        eng_vec = torch.LongTensor(eng_vec)

        # Vectorize Gujarati word
        guj_vec = [char2index[char] for char in word_pair[1]]
        guj_vec.append(EOS_token)
        guj_vec += [PAD_token] * (MAX_LENGTH - len(word_pair[1]))
        guj_vec = torch.LongTensor(guj_vec)

        vec_pair = (eng_vec, guj_vec)
        vec_pair_list.append(vec_pair)

    return vec_pair_list, char2index, index2char


In [ ]:
def train(encoder, decoder,train_loader, encoder_optimizer,decoder_optimizer,encoder_num_layers,decoder_num_layers,cell_type,criterion,char2index,index2char,MAX_LENGTH,teacher_forcing_ratio,attention, device):
    """
    Train the encoder-decoder model.

    Parameters:
        encoder (nn.Module): Encoder model.
        decoder (nn.Module): Decoder model.
        train_loader (DataLoader): DataLoader for training data.
        encoder_optimizer (torch.optim): Optimizer for encoder.
        decoder_optimizer (torch.optim): Optimizer for decoder.
        encoder_num_layers (int): Number of layers in the encoder.
        decoder_num_layers (int): Number of layers in the decoder.
        cell_type (str): Type of RNN cell used in the model.
        criterion (nn.Module): Loss criterion.
        char2index (dict): Dictionary mapping characters to indices.
        index2char (dict): Dictionary mapping indices to characters.
        MAX_LENGTH (int): Maximum length of input sequences.
        teacher_forcing_ratio (float): Probability of using teacher forcing during training.
        attention (bool): Whether to use attention mechanism.
        device (torch.device): Device to run the model on.

    Returns:
        final_loss (float): Average loss over the training data.
        accuracy (float): Accuracy of the model.
        atten_weights (Tensor): Attention weights.
    """

    total_loss = 0
    correct=0
    total=0
    encoder.to(device)
    decoder.to(device)
    encoder.train()
    decoder.train()
    target_length=0
    atten_weights = torch.zeros(1,MAX_LENGTH+1, MAX_LENGTH+1).to(device)

    # Iterate over the training data
    for data in tqdm(train_loader):

        # Get input and target tensors
        input_tensor, target_tensor = data
        input_tensor = input_tensor.to(device)
        target_tensor=target_tensor.to(device)
        batch_size = input_tensor.shape[0]
        input_tensor=input_tensor.T
        target_tensor=target_tensor.T

        # Initialize hidden state for encoder
        encoder_hidden = encoder.initHidden()

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        input_length = len(input_tensor)
        target_length = len(target_tensor)


        # Forward pass through encoder
        encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)


        decoder_input = target_tensor[0]

        # Handle different numbers of layers in the encoder and decoder
        if encoder_num_layers != decoder_num_layers:
            if encoder_num_layers < decoder_num_layers:
                remaining_layers = decoder_num_layers - encoder_num_layers
                # Copy all encoder hidden layers and then repeat the top layer
                if cell_type == "LSTM":
                    top_layer_hidden = (encoder_hidden[0][-1].unsqueeze(0), encoder_hidden[1][-1].unsqueeze(0))
                    extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                    decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=0), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=0))
                else:
                    top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_size)
                    extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                    decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=0)

            else:
                # Slice the hidden states of the encoder to match the decoder layers
                if cell_type == "LSTM":
                    decoder_hidden = (encoder_hidden[0][-decoder_num_layers:], encoder_hidden[1][-decoder_num_layers:])
                else :
                    decoder_hidden = encoder_hidden[-decoder_num_layers:]
        else:
            decoder_hidden = encoder_hidden


        loss = 0

        atten_weight_default = torch.zeros(batch_size,1, MAX_LENGTH + 1).to(device)

        pred=torch.zeros(len(target_tensor), batch_size).to(device)
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):

                if attention == True:
                    decoder_output, decoder_hidden, atten_weight = decoder(decoder_input, decoder_hidden, encoder_output)
                    atten_weight_default = torch.cat((atten_weight_default, atten_weight), dim = 1)

                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)


                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi=torch.squeeze(topi)
                pred[di]=topi

                decoder_input = target_tensor[di]


        else:
            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
            for di in range(target_length):

                if attention == True:
                    decoder_output, decoder_hidden, atten_weight = decoder(decoder_input, decoder_hidden, encoder_output)
                    atten_weight_default = torch.cat((atten_weight_default, atten_weight), dim = 1)

                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi = torch.squeeze(topi)
                pred[di]=topi
                decoder_input = topi


        if attention == True:
            atten_weights = torch.cat((atten_weights, atten_weight_default[:, 1:, :]), dim = 0)

        pred = pred.T
        act = target_tensor.T


        # Calculate accuracy
        for i in range(len(pred)):
            f=0
            for j in range(len(pred[i])):
                if(pred[i][j]!=act[i][j]):
                    f=1
                    break
            if(f==0):
                correct += 1

        total_loss += loss
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()



    accuracy = correct / (len(train_loader) * batch_size )
    final_loss = total_loss /  (len(train_loader) * batch_size )


    return   final_loss , accuracy,atten_weights[1:, :, :]




In [ ]:

# Define evaluation function
def evaluate(encoder, decoder, val_loader,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,attention, device):
    """
    Evaluate the encoder-decoder model.

    Parameters:
        encoder (nn.Module): Encoder model.
        decoder (nn.Module): Decoder model.
        val_loader (DataLoader): DataLoader for validation data.
        encoder_num_layers (int): Number of layers in the encoder.
        decoder_num_layers (int): Number of layers in the decoder.
        cell_type (str): Type of RNN cell used in the model.
        criterion (nn.Module): Loss criterion.
        char2index (dict): Dictionary mapping characters to indices.
        index2char (dict): Dictionary mapping indices to characters.
        MAX_LENGTH (int): Maximum length of input sequences.
        attention (bool): Whether to use attention mechanism.
        device (torch.device): Device to run the model on.

    Returns:
        final_loss (float): Average loss over the validation data.
        accuracy (float): Accuracy of the model.
        predictions (list): Predicted words.
        Input (list): Input words.
        Target (list): Target words.
        atten_weights (Tensor): Attention weights.
    """



    EOS_token=1
    SOS_token=0
    correct=0
    total_loss=0
    target_length=0
    pred=[]
    predictions = []
    Input = []
    Target = []
    atten_weights = torch.zeros(1,MAX_LENGTH+1, MAX_LENGTH+1).to(device)

    with torch.no_grad():

        encoder.eval()
        decoder.eval()
        for data in tqdm(val_loader):

            # Extract input and target tensors
            input_tensor, target_tensor = data
            batch_size = input_tensor.shape[0]
            input_tensor = input_tensor.to(device)
            target_tensor=target_tensor.to(device)

            # Transpose tensors for batch processing
            input_tensor=input_tensor.T
            target_tensor=target_tensor.T

            # Initialize encoder hidden state
            encoder_hidden = encoder.initHidden()

            input_length = len(input_tensor)
            target_length = len(target_tensor)

            # Forward pass through encoder
            encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)


            decoder_input = target_tensor[0]

            # Handle different numbers of layers in the encoder and decoder
            if encoder_num_layers != decoder_num_layers:
                if encoder_num_layers < decoder_num_layers:
                    remaining_layers = decoder_num_layers - encoder_num_layers
                    # Copy all encoder hidden layers and then repeat the top layer
                    if cell_type == "LSTM":
                        top_layer_hidden = (encoder_hidden[0][-1].unsqueeze(0), encoder_hidden[1][-1].unsqueeze(0))
                        extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                        decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=0), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=0))
                    else:
                        top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_size)
                        extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                        decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=0)

                else:
                    # Slice the hidden states of the encoder to match the decoder layers
                    if cell_type == "LSTM":
                        decoder_hidden = (encoder_hidden[0][-decoder_num_layers:], encoder_hidden[1][-decoder_num_layers:])
                    else :
                        decoder_hidden = encoder_hidden[-decoder_num_layers:]
            else:
                decoder_hidden = encoder_hidden


            loss = 0
            pred=torch.zeros(len(target_tensor), batch_size).to(device)
            atten_weight_default = torch.zeros(batch_size,1, MAX_LENGTH + 1).to(device)


            # Decode sequence
            for di in range(target_length):
                if attention == True:

                    decoder_output, decoder_hidden, atten_weight = decoder(decoder_input, decoder_hidden, encoder_output)
                    atten_weight_default = torch.cat((atten_weight_default, atten_weight), dim = 1)

                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi = torch.squeeze(topi)
                pred[di]=topi
                decoder_input = topi

            if attention == True:
                atten_weights = torch.cat((atten_weights, atten_weight_default[:, 1:, :]), dim = 0)

            pred = pred.T
            act = target_tensor.T
            act_eng = input_tensor.T

            # Collect predictions, inputs, and targets
            for i in range(batch_size):
                pred_word=""
                input_word=""
                target_word = ""

                f=0
                for j in range(len(act[i])):

                    if(int(pred[i][j].item()) > 2):
                        pred_word += index2char[int(pred[i][j].item())]
                    if(int(act[i][j].item()) > 2):
                        target_word += index2char[int(act[i][j].item())]

                    if(pred[i][j]!=act[i][j]):
                        f=1

                if(f==0):
                    correct += 1

                for j in range(len(act_eng[i])):

                      if(int(act_eng[i][j].item()) > 2):
                          input_word += index2char[int(act_eng[i][j].item())]

                predictions.append(pred_word)
                Input.append(input_word)
                Target.append(target_word)


        total_loss += loss
        accuracy = correct / (len(val_loader) * batch_size )
        final_loss = total_loss / (len(val_loader) * batch_size )
    return  final_loss , accuracy , predictions ,Input , Target ,atten_weights[1:, :, :]


In [ ]:

# Define training function
def arguments(input_embedding_size,encoder_num_layers,decoder_num_layers,hidden_size,cell_type,bidirectional,batch_size,learning_rate,num_epochs,dropout,teacher_forcing_ratio,attention,mode,pathofzip,Folder_name):


    """
    Define arguments and run the training or testing mode.

    Parameters:
        input_embedding_size (int): Size of input embeddings.
        encoder_num_layers (int): Number of layers in the encoder.
        decoder_num_layers (int): Number of layers in the decoder.
        hidden_size (int): Size of the hidden layer.
        cell_type (str): Type of RNN cell used in the model.
        bidirectional (bool): Whether the encoder is bidirectional.
        batch_size (int): Batch size for training.
        learning_rate (float): Learning rate for optimization.
        num_epochs (int): Number of training epochs.
        dropout (float): Dropout probability.
        teacher_forcing_ratio (float): Ratio of teacher forcing during training.
        attention (bool): Whether to use attention mechanism.
        mode (str): Training or Testing mode.
        pathofzip (str): Path to the zip file containing the dataset.
        Folder_name (str): Name of the folder containing the dataset.

    Returns:
        None
    """

    # Retrieve datasets
    train_dataset ,val_dataset ,test_dataset= Datasetretrival(pathofzip,Folder_name)

    # Convert DataFrame to list of tuples
    data_pairs_train = [list(row) for row in train_dataset.values]
    data_pairs_val = [list(row) for row in val_dataset.values]
    data_pairs_test = [list(row) for row in test_dataset.values]


    # Initialize index-char dictionaries
    index2char = {0:'<', 1: '>', 2 : '.'}
    char2index = {'<' : 0, '>' : 1, '.' : 2 }


    # Define maximum sequence length
    MAX_LENGTH= 27

    # Vectorize data
    vec_pair_list_train,char2index, index2char=vecorizeddata(data_pairs_train,index2char, char2index,MAX_LENGTH)
    vec_pair_list_val,_,_=vecorizeddata(data_pairs_val,index2char, char2index,MAX_LENGTH)
    vec_pair_list_test,_,_=vecorizeddata(data_pairs_test,index2char, char2index,MAX_LENGTH)



    # initialize prediction target and Input List
    predictions = []
    Input = []
    Target = []

    # Initialize device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    atten_weights = torch.zeros(1,MAX_LENGTH+1, MAX_LENGTH+1).to(device)

    # Create DataLoader objects
    train_loader = DataLoader(vec_pair_list_train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(vec_pair_list_val, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(vec_pair_list_test, batch_size=batch_size, shuffle=False)

    # Define model
    encoder = Encoder(len(char2index),input_embedding_size, hidden_size,batch_size, encoder_num_layers, cell_type,bidirectional, dropout).to(device)

    if attention ==True:
        decoder = AttentionDecoderRNN(hidden_size, len(char2index), input_embedding_size,  decoder_num_layers,cell_type, dropout, batch_size, MAX_LENGTH+1).to(device)
    else:
        decoder = Decoder(len(char2index),input_embedding_size, hidden_size,batch_size, decoder_num_layers, cell_type, dropout,MAX_LENGTH).to(device)



    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)


    # Training and Validation loop
    if(mode=='Normal'):

        for epoch in range(num_epochs):
            # Train the model
            train_loss, train_accuracy,_ = train(encoder,decoder, train_loader, encoder_optimizer,decoder_optimizer,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,teacher_forcing_ratio,attention, device)

            # Validation loop
            val_loss, val_accuracy,predictions ,Input , Target , _ = evaluate(encoder,decoder, val_loader,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,attention, device)

            # # Log metrics to Weights & Biases
            # wandb.log({
            #     "Epoch": epoch + 1,
            #     "Train_Accuracy": train_accuracy,
            #     "Train_Loss": train_loss,
            #     "Val_Accuracy": val_accuracy,
            #     "Val_Loss": val_loss
            # })

            # Print epoch results
            print(f"\nEpoch {epoch+1}/{num_epochs},\n Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f},\n Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}\n")


    # Training and Testing Loop
    elif(mode=='Test'):

        for epoch in range(num_epochs):
            # Train the model
            train_loss, train_accuracy,_ = train(encoder,decoder, train_loader, encoder_optimizer,decoder_optimizer,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,teacher_forcing_ratio,attention, device)

            # Test the model
            test_loss, test_accuracy,predictions ,Input , Target ,atten_weights  = evaluate(encoder,decoder, test_loader,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,attention, device)

            # # Log metrics to Weights & Biases
            # wandb.log({
            #     "Epoch": epoch + 1,
            #     "Train_Accuracy": train_accuracy,
            #     "Train_Loss": train_loss,
            #     "Val_Accuracy": val_accuracy,
            #     "Val_Loss": val_loss
            # })

            # Print epoch results
            print(f"\nEpoch {epoch+1}/{num_epochs},\n Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f},\n Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}\n")


        # Initialize wandb
        wandb.init(project="DL_Assignment_3_CS23M046", name="attn_visualization")

        # Define the grid dimensions
        rows = int(np.ceil(np.sqrt(10)))
        cols = int(np.ceil(10 / rows))




        for i in range(10):
            if i < 10:
                prediction = predictions[i]

                pred_word=""
                input_word=""

                for j in range(len(prediction)):
                    # Ignore padding
                    if(prediction[j] != '#'):
                        pred_word += prediction[j]
                    else :
                        break
                input_seq = data_pairs_test[i][0]



                for j in range(len(input_seq)):
                    if(input_seq[j] != '#'):
                        input_word += input_seq[j]
                    else :
                        break
                attn_weights = atten_weights[i, :len(pred_word), :len(input_word)].detach().cpu().numpy()

                # Create and log each heatmap individually
                x_labels = list(input_word)
                y_labels = list(pred_word)
                heatmap = wandb.plots.HeatMap(
                    x_labels=x_labels,
                    y_labels=y_labels,
                    matrix_values=attn_weights,
                    show_text=True
                )
                wandb.log({'heatmap_' + str(i + 1): heatmap}, commit=False)


        # Finish the wandb run
        wandb.finish()

    print(f"\n Predictions Generated by current sweep and actual output:\n")
    dataframe = pd.DataFrame({"INPUT": Input, "PREDICTED": predictions,"ACTUAL":Target})
    dataframe.to_csv("predictions.csv", index=False)
    data = pd.read_csv("predictions.csv",header=None)
    #files.download("predictions.csv")
    display(data)




In [ ]:
arguments(32,2,2,1024,"LSTM",True,128,0.001,2,0.2,0.5,True,'Test','/content/sample_data/aksharantar_sampled.zip','guj')

In [ ]:
wandb.login()

True

In [ ]:

# Define hyperparameters to sweep
sweep_config = {
    "method": "bayes",
    'name'  : 'Train Dataset Run',
    'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
    "parameters": {
        "input_embedding_size": {"values": [16, 32, 64, 256]},
        "encoder_num_layers": {"values": [1, 2, 3]},
        "decoder_num_layers": {"values": [1, 2, 3]},
        "hidden_size": {"values": [128,256,512,1024]},
        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},
        "bidirectional": {"values": [True, False]},
        "batch_size": {"values": [32, 64 , 128]},
        "learning_rate": {"values": [0.001, 0.0001]},
        "num_epochs": {"values": [5,10 ,15]},
        "dropout": {"values": [0.2, 0.3]},
        "teacher_forcing_ratio" : {"values":[0.5]},
        "attention": {"values": [False]},
        "mode": {"values": ["Normal","Test"]}
    }
}

# Initialize wandb sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="DL_Assignment_3_CS23M046")






Create sweep with ID: mtokz2ko
Sweep URL: https://wandb.ai/cs23m046/DL_Assignment_3_CS23M046/sweeps/mtokz2ko


In [ ]:
def main_1():

    # Initialize wandb
    with wandb.init() as run:

        config = wandb.config

        run_name=str(config.cell_type)+"_embedding_"+str(config.input_embedding_size)+"_hidden_size_"+str(config.hidden_size)+"_bidirectional_"+str(config.bidirectional)+"_Encoder_layers_"+str(config.encoder_num_layers)+"_Decoder_layers_"+str(config.decoder_num_layers)
        wandb.run.name=run_name

        pathofzip='/content/sample_data/aksharantar_sampled.zip'
        Folder_name='guj'

        arguments(config.input_embedding_size,config.encoder_num_layers,config.decoder_num_layers,config.hidden_size,config.cell_type,config.bidirectional,config.batch_size,config.learning_rate,config.num_epochs,config.dropout,config.teacher_forcing_ratio,config.attention,config.mode,pathofzip,Folder_name)



# Run sweep
wandb.agent(sweep_id, function=main_1, count=10)

wandb.finish()

wandb: Agent Starting Run: lrfylk5g with config:
wandb: 	attention: False
wandb: 	batch_size: 128
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	decoder_num_layers: 2
wandb: 	dropout: 0.3
wandb: 	encoder_num_layers: 3
wandb: 	hidden_size: 1024
wandb: 	input_embedding_size: 16
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 10
wandb: 	teacher_forcing_ratio: 0.5


 35%|███▌      | 141/400 [01:08<02:09,  1.99it/s]

 36%|███▋      | 145/400 [01:10<02:08,  1.99it/s]

In [ ]:
import argparse
import wandb
wandb.login()





DEFAULT_HYPERPARAMETERS = {

    'wandb_project': 'DL_Assignment_2_CS23M046',
    'wandb_entity': 'cs23m046',
    'Folder_path': None,
    'Folder_name' : None ,
    'input_embedding_size': [256],
    'encoder_num_layers':  [3],
    'decoder_num_layers': [3],
    'hidden_size':  [1024],
    'cell_type':  ['LSTM'],
    'bidirectional': [True],
    'batch_size': {'values': [128]},
    'learning_rate': {'values': [0.0001]},
    'num_epochs': {'values': [15]},
    'dropout':  [0.2],
    'teacher_forcing_ratio': [0.5],
    'attention': [True],
    'mode':  ['Normal']


}




def main():
    parser = argparse.ArgumentParser(description="RNN based seq2seq model")

    parser.add_argument("--wandb_project", type=str, default=DEFAULT_HYPERPARAMETERS['wandb_project'], help="Project name used to track experiments in Weights & Biases dashboard")
    parser.add_argument("--wandb_entity", type=str, default=DEFAULT_HYPERPARAMETERS['wandb_entity'], help="Wandb Entity used to track experiments in the Weights & Biases dashboard.")
    parser.add_argument("--Folder_path", type=str, default=DEFAULT_HYPERPARAMETERS['Folder_path'], help=" Folder Path")
    parser.add_argument("--Folder_name", type=str, default=DEFAULT_HYPERPARAMETERS['Folder_name'], help="Folder name of Language")
    parser.add_argument("--input_embedding_size", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['input_embedding_size'], help="Input embedding size")
    parser.add_argument("--encoder_num_layers", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['encoder_num_layers'], help="Number of layers in the encoder")
    parser.add_argument("--decoder_num_layers", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['decoder_num_layers'], help="Number of layers in the decoder")
    parser.add_argument("--hidden_size", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['hidden_size'], help="Hidden size of the model")
    parser.add_argument("--cell_type", type=str, nargs='+', default=DEFAULT_HYPERPARAMETERS['cell_type'], help="Type of cell used in the model")
    parser.add_argument("--bidirectional", type=bool, nargs='+', default=DEFAULT_HYPERPARAMETERS['bidirectional'], help="Whether to use bidirectional RNN")
    parser.add_argument("--batch_size", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['batch_size'], help="Batch size for training data")
    parser.add_argument("--learning_rate", type=float, nargs='+', default=DEFAULT_HYPERPARAMETERS['learning_rate'], help="Learning rate for model training")
    parser.add_argument("--num_epochs", type=int, nargs='+', default=DEFAULT_HYPERPARAMETERS['num_epochs'], help="Number of epochs for training")
    parser.add_argument("--dropout", type=float, nargs='+', default=DEFAULT_HYPERPARAMETERS['dropout'], help="Dropout rate to prevent overfitting")
    parser.add_argument("--teacher_forcing_ratio", type=float, nargs='+', default=DEFAULT_HYPERPARAMETERS['teacher_forcing_ratio'], help="Teacher forcing ratio")
    parser.add_argument("--attention", type=bool, nargs='+', default=DEFAULT_HYPERPARAMETERS['attention'], help="Whether to use attention mechanism")
    parser.add_argument("--mode", type=str, default=DEFAULT_HYPERPARAMETERS['mode'], help="Mode of operation")

    args = parser.parse_args()



    # Extracting individual hyperparameters
    wandb_project = args.wandb_project
    wandb_entity = args.wandb_entity
    Folder_path = args.Folder_path
    Folder_name = args.Folder_name
    input_embedding_size = args.input_embedding_size[0]
    encoder_num_layers = args.encoder_num_layers[0]
    decoder_num_layers = args.decoder_num_layers[0]
    hidden_size = args.hidden_size[0]
    cell_type = args.cell_type[0]
    bidirectional = args.bidirectional[0]
    batch_size = args.batch_size[0]
    learning_rate = args.learning_rate[0]
    num_epochs = args.num_epochs[0]
    dropout = args.dropout[0]
    teacher_forcing_ratio = args.teacher_forcing_ratio[0]
    attention = args.attention[0]
    mode = args.mode


    # Initialize wandb
    wandb.init(project=wandb_project, entity=wandb_entity)
    run_name=str(cell_type)+"_embedding_"+str(input_embedding_size)+"_hidden_size_"+str(hidden_size)+"_bidirectional_"+str(bidirectional)+"_Encoder_layers_"+str(encoder_num_layers)+"_Decoder_layers_"+str(decoder_num_layers)
    wandb.run.name = run_name


    arguments(input_embedding_size,encoder_num_layers,decoder_num_layers,hidden_size,cell_type,bidirectional,batch_size,learning_rate,num_epochs,dropout,teacher_forcing_ratio,attention,mode,Folder_path,Folder_name)

    wandb.finish()

if __name__ == "__main__":
    main()